In [100]:
# Initiation
# importing required libraries
import os
import glob

import numpy as np
import pandas as pd

In [101]:
path = os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy"
print(path)

/home/jupyter/Team-Prophecy


In [102]:
if path != os.getcwd():
    os.chdir(path)

In [103]:
from Code.src.modules.db_ops import *

Be sure to incorporate buildEnrollmentData as a function for all lines below

In [104]:
#def buildEnrollmentData():

In [105]:
# # Enrollment Data
print("#"*40, "\t", "\n", "Enrollment Data")
print("Building the Enrollment Data... [1/3]")
# Merging the Enrollment data from all the CSV files in the EnrollmentData folder
files = glob.glob(
    os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data", "01_raw", "EnrollmentData", "*.csv")
)

######################################## 	 
 Enrollment Data
Building the Enrollment Data... [1/3]


In [106]:
df_enrollment = pd.read_csv(files[0])
df_enrollment['file_name'] = files[0]
for file in files[1:]:
    data = pd.read_csv(file)
    data['file_name'] = file
    df_enrollment = pd.concat([df_enrollment, data], ignore_index=True)

# Shape of the final dataframe
df_enrollment.shape

# renaming all the columns in the dataframe
rename_dict = {
    'Unnamed: 0'                        : 'file_index',
    'Stu Term'                          : 'reg_term_code',
    'Unnamed: 1'                        : 'reg_term_desc',
    'Stu Admit Term'                    : 'stu_admit_term_code',
    'Unnamed: 3'                        : 'stu_admit_term_desc',
    'Record ID'                         : 'stu_id',
    'Stu Visa Type'                     : 'stu_visa',
    'Stu Attribute BAM'                 : 'stu_bam',
    'Stu Admit Type'                    : 'stu_prog_level',
    'Stu Primary Degree Level'          : 'stu_deg_level',
    'Stu Primary Major 1 College'       : 'stu_college',
    'Stu Primary Program'               : 'stu_prog_desc',
    'Stu Primary Program Code'          : 'stu_prog_code',
    'Stu Primary Department'            : 'stu_dept',
    'Unnamed: 14'                       : 'stu_dept_desc',
    'Stu New/Returning Ind'             : 'stu_new_ret',
    'Stu Residency Group'               : 'stu_res',
    'Course Sect College'               : 'crs_sect_clg',
    'Stu Registered Ind'                : 'stu_act_reg_ind',
    'Registration Status'               : 'reg_status',
    'Registration Status Date'          : 'reg_status_date',
    'Course Desc'                       : 'crs',
    'Course Section'                    : 'crs_sect',
    'Course Sect Schedule Type'         : 'crs_type',
    'Course Sect Wiley Courses Ind'     : 'crs_sect_wiley_ind',
    'Course Sect Credits'               : 'crs_credits',
    'Stu Course Registered Hours'       : 'crs_hours',
    'Course Sect Instruction Delivery Method Group' : 'crs_sect_modality',
}
df_enrollment.rename(columns=rename_dict, inplace=True)

In [107]:
df_enrollment['reg_term_name'] = df_enrollment['reg_term_desc'].str.split(' ').str[0]
df_enrollment['reg_term_year'] = df_enrollment['reg_term_desc'].str.split(' ').str[1]
# splitting the "stu_admit_term_desc" column into term and year
df_enrollment['stu_admit_term_name'] = df_enrollment['stu_admit_term_desc'].str.split(' ').str[0]
df_enrollment['stu_admit_term_year'] = df_enrollment['stu_admit_term_desc'].str.split(' ').str[1]

# storing the Record Extraction Date from the "file_name" column
df_enrollment['rec_ext_date'] = df_enrollment['file_name'].str.split("_").str[-2]
df_enrollment['rec_ext_date'] = df_enrollment['rec_ext_date'].str.replace(".", "/", regex=False)

# Cleaning the data
print("Cleaning the Enrollment Data... [2/3]")
df_enrollment['stu_id'].dropna(inplace=True)
df_enrollment['stu_visa'].fillna("Not Relevent", inplace=True)

df_enrollment['stu_bam'].replace("'--", "Not BAM", inplace=True)
df_enrollment['stu_dept'].replace("'-----", "No Value", inplace=True)
df_enrollment['crs_sect_wiley_ind'].replace("'--", "No Value", inplace=True)

df_enrollment['crs_credits'] = df_enrollment['crs_credits'].astype(str)

df_enrollment['crs_credits'].replace("6-Jan", "1-6", inplace=True)
df_enrollment['crs_credits'].replace("3-Jan", "1-3", inplace=True)
df_enrollment['crs_credits'].replace("18-Jan", "1-18", inplace=True)
df_enrollment['crs_credits'].replace("4-Jan", "1-4", inplace=True)
df_enrollment['crs_credits'].replace("0,3", "0-3", inplace=True)

# sorting the dataframe by term code and registration status date
df_enrollment.sort_values(
    by=['reg_term_code', 'rec_ext_date', 'reg_status_date'], inplace=True
)

# resetting the index after sorting
df_enrollment.reset_index(drop=True, inplace=True)

# creating a new rec_id column for identifying each record
df_enrollment['rec_id'] = df_enrollment.index

# Changing the data type of the columns
df_enrollment['rec_ext_date']           = pd.to_datetime(df_enrollment['rec_ext_date'])
df_enrollment['reg_term_code']          = df_enrollment['reg_term_code'].astype(str)
df_enrollment['reg_term_year']          = df_enrollment['reg_term_year'].astype(int)
df_enrollment['reg_term_name']          = df_enrollment['reg_term_name'].astype('category')
df_enrollment['stu_new_ret']            = df_enrollment['stu_new_ret'].astype('category')
df_enrollment['stu_deg_level']          = df_enrollment['stu_deg_level'].astype('category')
df_enrollment['stu_college']            = df_enrollment['stu_college'].astype('category')
df_enrollment['stu_prog_desc']          = df_enrollment['stu_prog_desc'].astype('category')
df_enrollment['stu_prog_level']         = df_enrollment['stu_prog_level'].astype('category')
df_enrollment['stu_dept']               = df_enrollment['stu_dept'].astype('category')
df_enrollment['stu_dept_desc']          = df_enrollment['stu_dept_desc'].astype('category')
df_enrollment['stu_admit_term_code']    = df_enrollment['stu_admit_term_code'].astype(str)
df_enrollment['stu_admit_term_year']    = df_enrollment['stu_admit_term_year'].astype(int)
df_enrollment['stu_admit_term_name']    = df_enrollment['stu_admit_term_name'].astype('category')
df_enrollment['stu_res']                = df_enrollment['stu_res'].astype('category')
df_enrollment['stu_visa']               = df_enrollment['stu_visa'].astype('category')
df_enrollment['stu_bam']                = df_enrollment['stu_bam'].astype('category')
df_enrollment['crs_sect_clg']           = df_enrollment['crs_sect_clg'].astype('category')
df_enrollment['crs_type']               = df_enrollment['crs_type'].astype('category')
df_enrollment['crs_sect_modality']      = df_enrollment['crs_sect_modality'].astype('category')
df_enrollment['crs_sect_wiley_ind']     = df_enrollment['crs_sect_wiley_ind'].astype('category')
df_enrollment['crs_credits']            = df_enrollment['crs_credits'].astype('category')
df_enrollment['crs_hours']              = df_enrollment['crs_hours'].astype('category')
df_enrollment['stu_act_reg_ind']        = df_enrollment['stu_act_reg_ind'].astype('category')
df_enrollment['reg_status']             = df_enrollment['reg_status'].astype('category')
df_enrollment['reg_status_date']        = pd.to_datetime(df_enrollment['reg_status_date'])

# reordering the columns
df_enrollment = df_enrollment[[
    # Records Info
    'rec_id', 'rec_ext_date', 'file_name', 'file_index',
    # Registration Term/Semester Info
    'reg_term_code', 'reg_term_year', 'reg_term_name', 'reg_term_desc',
    # Student Info
    'stu_id', 'stu_deg_level', 'stu_college', 'stu_res', 'stu_visa', 'stu_bam', 'stu_new_ret',
    'stu_dept', 'stu_dept_desc', 'stu_prog_code', 'stu_prog_level', 'stu_prog_desc',
    'stu_admit_term_code', 'stu_admit_term_year', 'stu_admit_term_name', 'stu_admit_term_desc',
    # Course Info
    'crs', 'crs_type', 'crs_credits', 'crs_hours',
    'crs_sect', 'crs_sect_clg', 'crs_sect_modality', 'crs_sect_wiley_ind',
    # Registration Status Info
    'reg_status', 'reg_status_date', 'stu_act_reg_ind'
]]

print("Exporting the Enrollment Data... [3/3]")
raw_connection = ConnectDB(os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data", "01_raw", "raw.db"))
process_connection = ConnectDB(os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data", "02_processed", "intermediate.db"))

Cleaning the Enrollment Data... [2/3]
Exporting the Enrollment Data... [3/3]


In [131]:
df_enrollment.loc[:,['crs', 'crs_type', 'crs_credits', 'crs_hours','crs_sect', 'crs_sect_clg', 'crs_sect_modality', 'crs_sect_wiley_ind']].head()

,crs,crs_type,crs_credits,crs_hours,crs_sect,crs_sect_clg,crs_sect_modality,crs_sect_wiley_ind
0,INFS 640,Lecture,3,3.0,INFS 640 001,VSE,F2F,No Value
1,SWE 619,Lecture,3,3.0,SWE 619 002,VSE,F2F,No Value
2,CEIE 639,Lecture,3,3.0,CEIE 639 001,VSE,F2F,No Value
3,CEIE 679,Lecture,3,3.0,CEIE 679 001,VSE,F2F,No Value
4,SWE 645,Lecture,3,3.0,SWE 645 001,VSE,F2F,No Value


In [147]:
df_enrollment['crs'] = df_enrollment['crs'].str.replace(" ","")

In [150]:
df_enrollment['crs_sect'] = df_enrollment['crs_sect'].str.extract(r"\s(...)$")

In [108]:
df_enrollment["stu_deg_level"] = df_enrollment["stu_deg_level"].replace("Master","ms").replace("Graduate Certificate","gs").replace("PhD","phd")

In [201]:
df_enrollment["reg_status"] = df_enrollment["reg_status"].replace("**Web Registered**","R")\
                        .replace("**Registered**","R")\
                        .replace("Wait Listed","W")\
                        .replace("Web Withdrawal","D")\
                        .replace("Web Drop (Liability)","D")\
                        .replace("Drop/Delete","D")\
                        .replace("Drop-Course Cancelled","D")\
                        .replace("Withdrawal from Course", "D")\
                        .replace("Selective Withdrawal Exception", "D")\
                        .replace("Registered for Audit","R")

In [109]:
#df_enrollment.loc[:,["stu_id", "stu_admit_term_code","stu_deg_level","stu_college","stu_dept","stu_prog_code","stu_res","stu_visa","stu_bam"]]

THE INFORMATION WILL BE PLACED INTO THE INTERMEDIATE DB

In [202]:
reg_status_right = df_enrollment.loc[:,["stu_act_reg_ind","reg_term_code", "stu_id", "crs_sect", "reg_status_date", "reg_status","rec_id","stu_new_ret"]]
reg_status = df_enrollment.groupby(["reg_term_code", "stu_id", "crs_sect"]).agg({"reg_status_date":[np.max]}).reset_index()
reg_status.columns = ["reg_term_code", "stu_id", "crs_sect", "reg_status_date"]
reg_status = reg_status.merge(reg_status_right,on=["reg_term_code", "stu_id", "crs_sect","reg_status_date"],how="inner")
ret_status = reg_status[["reg_term_code","stu_id","crs_sect","reg_status_date","reg_status","stu_act_reg_ind","stu_new_ret"]]

In [203]:
ret_status["reg_status"].drop_duplicates()

0      R
26     D
794    W
Name: reg_status, dtype: category
Categories (3, object): ['R', 'W', 'D']

In [167]:
reg_status.columns

Index(['reg_term_code', 'stu_id', 'crs_sect', 'reg_status_date',
       'stu_act_reg_ind', 'reg_status', 'rec_id', 'stu_new_ret'],
      dtype='object')

reg_status is good
dept_list is good
reg_log is good
crs is good (but requires to be merged)
term_data is good

In [112]:
#Grab record_id, rec_ext_date
dept_list = list(df_enrollment.loc[:,["stu_dept","stu_college"]].drop_duplicates().itertuples(index=False, name=None))
dept_list = [i for i in dept_list if i[0] != "No Value"]
student_data = list(df_enrollment.loc[:,["stu_id", "stu_admit_term_code","stu_deg_level","stu_college","stu_dept","stu_prog_code","stu_res","stu_visa","stu_bam"]].itertuples(index=False, name=None))
reg_log_data = list(df_enrollment.loc[:,["rec_id","rec_ext_date","file_name","file_index"]].itertuples(index=False, name=None))
term_data = list(df_enrollment.loc[:,["reg_term_code","reg_term_year","reg_term_name","reg_term_desc"]].drop_duplicates().itertuples(index=False, name=None))

In [113]:
crs_data = list(df_enrollment.loc[:,["crs","crs_type","crs_credits","crs_hours"]].itertuples(index=False, name=None))

NOTE: student data is only being placed into a raw data connection because it has many referential database information that must be addressed prior

In [98]:
process_connection.connection.executemany("INSERT INTO departments VALUES(?,?)",dept_list)
process_connection.connection.commit()

process_connection.connection.executemany(
    "INSERT INTO registration_log(?,?)",
    reg_log_data)

raw_connection.executemany("""
INSERT INTO 
student_details(stu_admit_term_code,stu_deg_level,
                stu_college,stu_dept,stu_prog, stu_res,
                stu_visa, stu_bam)
VALUES(?,?,?,?,?,?,?,?)
""", student_data)
raw_connection.commit()



THE BELOW INFORMATION WILL BE PUT INTO THE RAW DB FOR FURTHER MERGING

In [26]:
col_list = df_enrollment.loc[:,["stu_college"]].drop_duplicates()